# Podstawy Sztucznej Inteligencji 2020/2021


Prosze uzupelnic kod tam gdzie znajduje napis `YOUR CODE HERE` lub 'YOUR ANSWER HERE'.

Warto zresetowac 'kernel' i sprawdzic czy caly notatnik uruchamiany od poczatku nie daje bledow.

---

# Backpropagation w zapisie tensorowym (z numpy)

Mam sieć jak składającą się z trzech  warstw: 
 - 3,4,2,3, 
 - aktywacja - sigmoid, 
 - funkcją straty jest błąd średniokwadratowy.

Obliczymy pochodne po dowolnej wadze lub bias-ie w dwóch przypadkach:

 - pojedyńczego przykładu
 - wielu przykłdów (batch)

In [59]:
import numpy as np 

W1,b1,W2,b2,W3,b3 = [np.array([[0.1, 0.1, 0.1, 0.1],
        [-0.1, 0.1, 0.2, 0.1],
        [0.1, 0.2, 0.1, 0.1]]),
 np.array([0.2, -0.2, 0.2, 0.2]),
 np.array([[-0.2, 0.2],
        [0.2, 0.2],
        [0.2, -0.2],
        [0.12, 0.2]]),
 np.array([0.4, 0.4]),
 np.array([[-0.3, 0.3, 0.13],
        [0.3, 0.3, 0.3]]),
 np.array([0.16, -0.6, 0.6] )]

s = lambda x:1/(1+np.exp(-x))

## 1. Przetwarzanie pojedyńczego przykładu

In [60]:
X = np.array([1,.3,.2])
y_ = np.array([.1,.2,.1])

### Forward pass

$$
z^1 = X^1 \cdot W^1 + b^1\\
y^1 = \sigma(z^1)
$$

$$
z^2 = y^1\cdot W^2 + b^2\\
y^2 = \sigma(z^2)
$$

$$
z^3 = y^2\cdot W^3 + b^3\\
y^3 = \sigma(z^3)
$$


In [61]:
z1 = X.dot(W1) + b1
y1 = s(z1)

z2 = y1.dot(W2) + b2
y2 = s(z2)

z3 = y2.dot(W3) + b3
y3 = s(z3)

y3

array([0.54056717, 0.44678642, 0.70632849])

### Backward pass

Propagujemy błąd (czułość) od ostatniej warstwy do pierwszej.

Przypuśćmy, że chcemy obliczyć:

$$\frac{\partial L}{\partial \mathbf{w^1}}$$



$$\frac{\partial L}{\partial w^1_{\alpha\beta}}=
\underbrace{\frac{2}{N^3}(y^3_i-\hat y_i) 
\frac{\partial y^3_i}{\partial z^3_i}  \frac{\partial z^3_i}{\partial y^2_j}\cdot
\frac{\partial y^2_j}{\partial z^2_j}  \frac{\partial z^2_j}{\partial y^1_k}\cdot
  }_{\Delta^1_k}\frac{\partial y^1_k}{\partial z^1_k} x^1_{\alpha} \delta_{k\beta}  
$$

$$\frac{\partial L}{\partial w^1_{\alpha\beta}}=
 \sum_k \Delta^1_k \frac{\partial y^1_k}{\partial z^1_k}  x^1_{\alpha} \delta_{k\beta}   = x^1_{\alpha} \frac{\partial y^1_\beta}{\partial z^1_\beta}  \Delta^1_\beta 
$$

$$\frac{\partial L}{\partial \mathbf{w^1}}= \underbrace{\mathbf{x^1}}_{n^1}\otimes \underbrace{\mathbf{\Delta^1  \mathbf{\sigma'}}}_{n^1}
$$



1.  Obliczamy czułość wyniku na aktywację ostatniej warstwy:
$$
\mathrm{error_i} =\frac{2}{N^3}(y^3_i-\hat y_i)  
$$





In [62]:
error = 2/(y3.shape[-1])*(y3-y_)
error

array([0.29371145, 0.16452428, 0.40421899])

In [63]:
y3.shape[-1]

3

$$
\mathrm{error_j} \to \underbrace{\frac{2}{N^3}(y^3_i-\hat y_i)}_{\mathrm{error_i}}  
\underbrace{\frac{\partial y^3_i}{\partial z^3_i}  \frac{\partial z^3_i}{\partial y^2_j}}_{M(n_3\times n_2)}= \mathrm{error_i}\frac{\partial y^3_i}{\partial z^3_i} \cdot
(w^{3\,T})_{ij}$$
$$
\frac{\partial z^3_i}{\partial y^2_j} =  w^3_{ji} = (w^{3\,T})_{ij}
$$

błąd z wyjscia warstwy 3 propagujemy wejscia warstwy 3

In [64]:
error = (y3*(1-y3)*error).dot(W3.T)
error

array([0.00121626, 0.05923688])


$$
\mathrm{error_j} \to   
= \mathrm{error_i}\frac{\partial y^2_i}{\partial z^2_i} \cdot
(w^{2\,T})_{ij}$$

błąd z wyjscia warstwy 2 propagujemy wejscia warstwy 1


In [65]:
error = (y2*(1-y2)*error).dot(W2.T)
error

array([ 0.00264531,  0.00275747, -0.00264531,  0.00273503])

Obliczamy pochodną po wagach  warstwy 3 jako iloczyn zewnętrzny wektora 
$$x^1_j$$
 oraz $$\mathrm{error_i} \frac{\partial y^1_i}{\partial z^1_i}$$



In [66]:
dw1 = np.outer(X, error*(y1*(1-y1)))
# dw1 = X[:,None] * (error*(y1*(1-y1)))[None, :]
dw1

array([[ 0.00064762,  0.00068921, -0.00063802,  0.00066324],
       [ 0.00019428,  0.00020676, -0.0001914 ,  0.00019897],
       [ 0.00012952,  0.00013784, -0.0001276 ,  0.00013265]])

In [67]:
np.testing.assert_allclose(dw1,[[0.0006476156413555145,
  0.0006892113015055656,
  -0.0006380165577866137,
  0.0006632386357523501],
 [0.0001942847011378035,
  0.00020676340500358492,
  -0.0001914049789775163,
  0.00019897159654647112],
 [0.000129523134091869,
  0.00013784226030111313,
  -0.00012760331446770579,
  0.00013264773588161916]] ,rtol=1e-3)

Pochodna po bias-ie wynosi:

$$\mathrm{error_i} \frac{\partial y^1_i}{\partial z^1_i}$$

In [69]:
db1 = error*(y1*(1-y1))
db1

array([ 0.00064762,  0.00068921, -0.00063802,  0.00066324])

In [70]:
np.testing.assert_allclose(db1,[0.0006476156413555145,
 0.0006892113015055656,
 -0.0006380165577866137,
 0.0006632386357523501], rtol=1e-3)

## 2. Batch processing

Co jeśli mamy wiele przykładów?

In [71]:
X = np.array([[1,.3,.2],
              [.1,.2,.1]])
y_ = np.array([[.1,.2,.1],
               [.2,.1,.2]])



### Forward pass

Zastosujmy te same wzory:

In [72]:
z1 = X.dot(W1) + b1
y1 = s(z1)

z2 = y1.dot(W2) + b2
y2 = s(z2)

z3 = y2.dot(W3) + b3
y3 = s(z3)

y3

array([[0.54056717, 0.44678642, 0.70632849],
       [0.54058069, 0.44643596, 0.70612085]])

### Backward pass

In [73]:
error = 2/(y3.shape[-1])*(y3-y_)
error

array([[0.29371145, 0.16452428, 0.40421899],
       [0.2270538 , 0.23095731, 0.3374139 ]])

$$
\mathrm{error_j} \to \underbrace{\frac{2}{N^3}(y^3_i-\hat y_i)}_{\mathrm{error_i}}  
\underbrace{\frac{\partial y^3_i}{\partial z^3_i}  \frac{\partial z^3_i}{\partial y^2_j}}_{M(n_3\times n_2)}= \mathrm{error_i}\frac{\partial y^3_i}{\partial z^3_i} \cdot
(w^{3\,T})_{ij}$$
$$
\frac{\partial z^3_i}{\partial y^2_j} =  w^3_{ji} = (w^{3\,T})_{ij}
$$

błąd z wyjscia warstwy 3 propagujemy wejscia warstwy 3


In [74]:
error = (y3 * (1-y3) * error).dot(W3.T)
error

array([[0.00121626, 0.05923688],
       [0.00930851, 0.05504532]])


$$
\mathrm{error_j} \to   
= \mathrm{error_i}\frac{\partial y^2_i}{\partial z^2_i} \cdot
(w^{2\,T})_{ij}$$

 error z wyjscia warstwy 2 propagujemy wejscia warsty 1

In [75]:
error = (y2 * (1-y2) * error).dot(W2.T)
error

array([[ 0.00264531,  0.00275747, -0.00264531,  0.00273503],
       [ 0.00208715,  0.00294805, -0.00208715,  0.00277587]])

Obliczamy pochodną po wagach  warstwy 3 jako iloczyn zewnętrzny wektora 
$$x^1_j$$
 oraz $$\mathrm{error_i} \frac{\partial y^1_i}{\partial z^1_i}$$

Jeśli zastosujemy reguły boadcastingu do obliczania iloczynu zewnętrzego, to otrzymamy wartości dla każdego przykładu z osobna:


In [76]:
dw1 = X[:,:,None]*( error*(y1*(1-y1)) )[:,None,:]
dw1.shape

(2, 3, 4)

Aby uzystać pochodną po wszytkich wartościach wag w tej warstwie należy uśrednić wynik po przykładach (czyli osi zerowej).

In [77]:
dw1 = np.mean(dw1, axis=0)
dw1

array([[ 3.49637974e-04,  3.81249771e-04, -3.44661610e-04,
         3.65822827e-04],
       [ 1.48802762e-04,  1.76669974e-04, -1.47009249e-04,
         1.67892797e-04],
       [ 9.05917690e-05,  1.05565268e-04, -8.94550355e-05,
         1.00527365e-04]])

In [78]:
np.testing.assert_allclose(dw1,[[0.0003496380231808871,
   0.00038124978891573846,
   -0.0003446616174187511,
   0.00036582283792085946],
  [0.0001488027919549495,
   0.00017666998610366136,
   -0.00014700925385113806,
   0.00016789280925877392],
  [9.059178410097957e-05,
   0.0001055652683135122,
   -8.945503941504285e-05,
   0.00010052736615762115]] ,rtol=1e-3)

Pochodna po bias-ie wynosi:

$$\mathrm{error_i} \frac{\partial y^1_i}{\partial z^1_i}$$

In [79]:
db1 = error*(y1*(1-y1))
db1 = np.mean(db1, axis=0)
db1

array([ 0.00058211,  0.00071105, -0.00057554,  0.00067365])

In [80]:
np.testing.assert_allclose(db1,[0.0005821100203320384,
  0.0007110470905900002,
  -0.0005755421589128673,
  0.0006736543728038669], rtol=1e-3)

## Dodatek iloczyn zewnętrzny

$$P_{ij} =  a_i b_j$$

In [81]:
a = np.array([1,2,3])
b = np.array([5,4])

In [82]:
np.outer(a,b)

array([[ 5,  4],
       [10,  8],
       [15, 12]])

In [83]:
a[:,None]*b[None,:]

array([[ 5,  4],
       [10,  8],
       [15, 12]])

In [28]:
[[a[i]*b[j] for j in range(b.shape[0])] for i in range(a.shape[0])] 

[[5, 4], [10, 8], [15, 12]]